In [1]:
""" This module prepares midi file data and feeds it to the neural
    network for training """
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

def train_network(cell_units = 512, keep_prob = 0.5, activation_func = 'softmax', loss_func = 'categorical_crossentropy', opt_func = 'rmsprop', sequence_len = 100):
    """ Train a Neural Network to generate music """
    all_notes = get_notes()
    # get amount of pitch names
    n_voc = len(set(all_notes))

    net_input, network_output = prepare_sequences(notes = all_notes, n_vocab = n_voc, sequence_length = sequence_len)

    model = create_network(network_input = net_input, n_vocab = n_voc, cell_units = cell_units, keep_prob = keep_prob, activation_func = activation_func, loss_func = loss_func, opt_func = opt_func)
    
    return model, net_input, network_output

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("Midi/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

def prepare_sequences(notes, n_vocab, sequence_length = 100):
    """ Prepare the sequences used by the Neural Network """
    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab, cell_units = 64, keep_prob = 0.5, activation_func = 'softmax', loss_func = 'categorical_crossentropy', opt_func = 'rmsprop'):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        cell_units,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(keep_prob))
    model.add(LSTM(cell_units, return_sequences=True))    
    model.add(Dropout(keep_prob))
    model.add(LSTM(cell_units))
    model.add(Dense((int)(cell_units/2)))
    model.add(Dropout(keep_prob))
    model.add(Dense(n_vocab))

    
    model.add(Activation(activation_func))
    model.compile(loss=loss_func, optimizer=opt_func)

    return model

def train(model, network_input, network_output, ep = 20):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=ep, batch_size=64, callbacks=callbacks_list)


Using TensorFlow backend.


In [2]:
""" This module generates notes for a midi file using the
    trained neural network """
import pickle
import numpy
from music21 import instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation

def generate(model_name, sequence_len = 100, n_len = 500):
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences_out(notes, pitchnames, n_vocab, sequence_length = sequence_len)
    model = create_network_out(normalized_input, n_vocab, model_name, cell_units = 512)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab, note_len = n_len)
    create_midi(prediction_output)

def prepare_sequences_out(notes, pitchnames, n_vocab, sequence_length = 100):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    #sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def create_network_out(network_input, n_vocab, model_name, cell_units = 64, keep_prob = 1.0, activation_func = 'softmax', loss_func = 'categorical_crossentropy', opt_func = 'rmsprop'):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        cell_units,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(keep_prob))
    model.add(LSTM(cell_units, return_sequences=True))
    model.add(Dropout(keep_prob))
    model.add(LSTM(cell_units))
    model.add(Dense((int)(cell_units/2)))
    model.add(Dropout(keep_prob))
    model.add(Dense(n_vocab))

    
    model.add(Activation(activation_func))
    model.compile(loss=loss_func, optimizer=opt_func)
    
    # Load the weights to each node
    model.load_weights(model_name)
    
    return model




def generate_notes(model, network_input, pitchnames, n_vocab, note_len = 500):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(note_len):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')



In [3]:
model,  network_input, network_output = train_network(
    keep_prob = 1.0, 
    opt_func = 'Adam', 
    sequence_len = 1000
)


Parsing Midi\darude-sandstorm.mid


W1113 01:06:40.701253  2592 deprecation_wrapper.py:119] From C:\Users\zjwar\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1113 01:06:40.760313  2592 deprecation_wrapper.py:119] From C:\Users\zjwar\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1113 01:06:40.775271  2592 deprecation_wrapper.py:119] From C:\Users\zjwar\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1113 01:06:42.412964  2592 deprecation_wrapper.py:119] From C:\Users\zjwar\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1113 01:06:42.445266  2592 deprecation_wrapper.py:119] From C:\Users\zjwar\Anac

In [8]:
model,  network_input, network_output = train_network(
    keep_prob = 1.0, 
    opt_func = 'Adam', 
    sequence_len = 100
)
#model.load_weights('weights-improvement-01-2.3777-bigger.hdf5')
model = load_model('Models\LSTM_Harder_Better_Faster_SandStorm.h5')
train(model, network_input, network_output, ep = 1)


Parsing Midi\darude-sandstorm.mid
Parsing Midi\HarderBetterFasterStronger.mid
Epoch 1/1
5162/5162 [==============================] - 247s 48ms/step - loss: 0.1962


In [5]:
model.save('Models\LSTM_Harder_Better_Faster_SandStorm.h5')

In [6]:
generate('Models\LSTM_Harder_Better_Faster_SandStorm.h5', sequence_len = 100)